# **Data Preprocessing**

In [0]:
import pandas as pd
import nltk
import gensim
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim.parsing.preprocessing import STOPWORDS
import numpy as np
import pickle
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
#mounting the google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#importing the CSV file of the dataset from google drive into a dataframe
data=pd.read_csv('drive/My Drive/duplicate_detection/eclipse_platform.csv')
data

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,1,P3,Team,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt fi...,CLOSED,FIXED,2,2001-10-10 21:34,2012-02-09 15:57
1,2,P5,Team,NaN,Opening repository resources doesnt honor type (1GE6I1Q),Opening repository resource always open the default text editor and doesnt; \thonor any mapping between resource types and editors.; \tAs a result it is not possible to view the contents of an image (*.gif file) in a sensible way.; ; NOTES:; EG (5/24/2001 5:13:45 AM); \tAgreed that this is needed eventually but not critical for June. \t; \tThere should also be a Java outliner when editing a java resource from the repository.; \tMoving to VCM.; ; KM (5/24/01 2:09:21 PM); \tIn general you cann...,RESOLVED,FIXED,2,2001-10-10 21:34,2002-05-07 10:33
2,3,P5,Team,NaN,Sync does not indicate deletion (1GIEN83),KM (10/2/2001 5:55:18 PM); \tThis PR about the deletion indicator in the sync viewer being too subtle.; \tThis would be even more true for someone with vision problems.; \tNE has good suggestion of using the label of the compare text viewer.; \tWe may also consider a better icon.; \tOn to the PR....; ; - delete file from project; - synchronize; - there is no indication that the file has been deleted from the workspace; - it looks the same as if the file is in the workspace but is empty; ; NO...,RESOLVED,FIXED,2,2001-10-10 21:34,2010-05-07 10:28
3,4,P5,Team,NaN,need better error message if catching up over read-only resource (1GF69TF),- become synchronized with some project in a repository; - use a different Eclipse to make a change to a file resource within this project and release it to the repository; - in the original Eclipse mark this file resource as being read-only (select it; right-click -> Properties; change; OK); - select the file resource; right-click -> Team -> Synchronize with Stream; - in the subsequent comparison view select the file resource; right-click -> Catchup; - since it has been marked as read-only ...,RESOLVED,FIXED,2,2001-10-10 21:34,2002-03-01 16:27
4,5,P3,Team,NaN,ISharingManager sharing API inconsistent (1GAUL8H),For getting/setting the managed state of a resource; the methods are:; ; isManaged(resource) and; manage(resource); ; for getting/setting the ignore state; the methods are:; ; getIgnored(resource); setIgnored(resource); ; These should be made more consistent. I suggest renaming ignore methods:; ; isIgnored(resource) and; ignore(resource).; ; I think its good practice to not use get and set in method names unless its really; just a field accessor.; ; NOTES:,RESOLVED,WONTFIX,2,2001-10-10 21:34,2008-08-15 8:04
...,...,...,...,...,...,...,...,...,...,...,...
85151,424679,P3,IDE,390071.0,Crash on bootstarp,NaN,CLOSED,DUPLICATE,4.3,2013-12-26 20:12,2013-12-31 5:32
85152,424714,P3,UI,NaN,[GTK/Linux] Blank Windows with GTK3,I started testing 4.4 and noticed the editor windows; package explorer; etc (all my views) are blank. I can see line numbers at left; and if I open; e.g. the updater window; I can see the contents without any problems. Since this was my first time using eclipse on this system; I tried 4.3 and all views were displaying contents. Then I started 4.3 with SWT_GTK3=1 and all contents were gone. So this is something to do with GTK3 but I have no idea of what else should I use to debug the issue.,CLOSED,NDUPLICATE,4.4,2013-12-28 8:57,2014-01-04 13:54
85153,424722,P3,Website,NaN,software,NaN,RESOLVED,IN

In [0]:
#Removing the unuseful coloumns from the dataset
data = data.drop(columns=['Priority', 'Component', 'Status', 'Resolution', 'Version', 'Created_time', 'Resolved_time'], axis=1)
data

,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt fi...
1,2,NaN,Opening repository resources doesnt honor type (1GE6I1Q),Opening repository resource always open the default text editor and doesnt; \thonor any mapping between resource types and editors.; \tAs a result it is not possible to view the contents of an image (*.gif file) in a sensible way.; ; NOTES:; EG (5/24/2001 5:13:45 AM); \tAgreed that this is needed eventually but not critical for June. \t; \tThere should also be a Java outliner when editing a java resource from the repository.; \tMoving to VCM.; ; KM (5/24/01 2:09:21 PM); \tIn general you cann...
2,3,NaN,Sync does not indicate deletion (1GIEN83),KM (10/2/2001 5:55:18 PM); \tThis PR about the deletion indicator in the sync viewer being too subtle.; \tThis would be even more true for someone with vision problems.; \tNE has good suggestion of using the label of the compare text viewer.; \tWe may also consider a better icon.; \tOn to the PR....; ; - delete file from project; - synchronize; - there is no indication that the file has been deleted from the workspace; - it looks the same as if the file is in the workspace but is empty; ; NO...
3,4,NaN,need better error message if catching up over read-only resource (1GF69TF),- become synchronized with some project in a repository; - use a different Eclipse to make a change to a file resource within this project and release it to the repository; - in the original Eclipse mark this file resource as being read-only (select it; right-click -> Properties; change; OK); - select the file resource; right-click -> Team -> Synchronize with Stream; - in the subsequent comparison view select the file resource; right-click -> Catchup; - since it has been marked as read-only ...
4,5,NaN,ISharingManager sharing API inconsistent (1GAUL8H),For getting/setting the managed state of a resource; the methods are:; ; isManaged(resource) and; manage(resource); ; for getting/setting the ignore state; the methods are:; ; getIgnored(resource); setIgnored(resource); ; These should be made more consistent. I suggest renaming ignore methods:; ; isIgnored(resource) and; ignore(resource).; ; I think its good practice to not use get and set in method names unless its really; just a field accessor.; ; NOTES:
...,...,...,...,...
85151,424679,390071.0,Crash on bootstarp,NaN
85152,424714,NaN,[GTK/Linux] Blank Windows with GTK3,I started testing 4.4 and noticed the editor windows; package explorer; etc (all my views) are blank. I can see line numbers at left; and if I open; e.g. the updater window; I can see the contents without any problems. Since this was my first time using eclipse on this system; I tried 4.3 and all views were displaying contents. Then I started 4.3 with SWT_GTK3=1 and all contents were gone. So this is something to do with GTK3 but I have no idea of what else should I use to debug the issue.
85153,424722,NaN,software,NaN
85154,424764,NaN,Crash (MacOS) - getIvar,Process: eclipse [42412]; Path: /Applications/eclipse.app/Eclipse.app/Contents/MacOS/eclipse; Identifier: org.eclipse.sdk.ide; Version: 4.4.0 (4.4.0.I20131212-1600); Code Type: X86-64 (Native); Parent Process: ??? [1]; Responsible: eclipse [42412]; User ID: 501; ; Date/Time: 2013-12-30 13:35:05.349 -0800; OS Version: Mac OS X 10.9.1 (13B42); Report Version: 11; Anonymous UUID: A1947271-CB55-A569-0A50-6F98CD512F75; ; Sleep/Wake U...


In [0]:
#Printing the Coloumn names
data.keys()

Index(['Issue_id', 'Duplicated_issue', 'Title', 'Description'], dtype='object')

In [0]:
#Checking the shape of the data
data.shape

(85156, 4)


## **1. Text Cleaning**

  *   Removing Invalid Reports
  *   Removing Punctuations






In [0]:
#Number of empty Description reports
data['Description'].isnull().sum()

129

In [0]:
#Number of empty Title reports
data['Title'].isnull().sum()

0

In [0]:
#removing the rows with empty Description
data = data.dropna(axis=0, subset=['Description'])

In [0]:
pd.options.display.max_colwidth = 500

In [0]:
#Showing the fixed pattern used in many reports which doesn't have to do anything with the bug itself
# Eg: fixed in head, has been marked as read only
data.loc[[4]]

,Issue_id,Duplicated_issue,Title,Description
4,5,NaN,ISharingManager sharing API inconsistent (1GAUL8H),For getting/setting the managed state of a resource; the methods are:; ; isManaged(resource) and; manage(resource); ; for getting/setting the ignore state; the methods are:; ; getIgnored(resource); setIgnored(resource); ; These should be made more consistent. I suggest renaming ignore methods:; ; isIgnored(resource) and; ignore(resource).; ; I think its good practice to not use get and set in method names unless its really; just a field accessor.; ; NOTES:


In [0]:
#Removing the fixed patterns by replacing them with " "
data["Description"]= data["Description"].str.replace("fixed in HEAD", "", case = False)
data["Description"]= data["Description"].str.replace("has been marked as readonly", " ", case = False)

In [0]:
#Checking if it is been removed
data.loc[[4]]

,Issue_id,Duplicated_issue,Title,Description
4,5,NaN,ISharingManager sharing API inconsistent (1GAUL8H),For getting/setting the managed state of a resource; the methods are:; ; isManaged(resource) and; manage(resource); ; for getting/setting the ignore state; the methods are:; ; getIgnored(resource); setIgnored(resource); ; These should be made more consistent. I suggest renaming ignore methods:; ; isIgnored(resource) and; ignore(resource).; ; I think its good practice to not use get and set in method names unless its really; just a field accessor.; ; NOTES:


In [0]:
pd.options.display.max_colwidth = 50

In [0]:
#Text Cleaning round 1 (removing punctutions)
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\w*\f\w*', '', text)
    text = re.sub('\(.*?\)', '', text)
    text = re.sub('\[.*]\)', '', text)
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [0]:
data['Description'] = data['Description'].apply(clean_text_round1)


In [0]:
# Apply a second round of cleaning (removing punctuations)
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\t', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [0]:
data['Title'] = data['Title'].apply(clean_text_round2)
data['Description'] = data['Description'].apply(clean_text_round2)

## **2. Stop Word Removal, Tokenization and Lemmatization**

In [0]:

def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 5:
            result.append(lemmatize(token))
    return result

In [0]:
data['Title'] = data['Title'].map(preprocess)
data['Description'] = data['Description'].map(preprocess)

In [0]:
data.to_csv("drive/My Drive/duplicate_detection/preprocessed_data.csv")

## **Seperation of Master Reports and Duplicate Reports**

In [0]:
#Number of master_reports
data['Duplicated_issue'].isnull().sum()

70629

In [0]:
#Saving all the duplicate reports into a csv file as a testing set
duplicate_reports = data.dropna(axis=0, subset=['Duplicated_issue'])
duplicate_reports.reset_index(drop=True)
duplicate_reports.to_csv('drive/My Drive/duplicate_detection/duplicate_reports.csv')

In [0]:
#Seperating all the master reports into a dataframe
master_reports = data[data.isnull().any(axis=1)]
master_reports.reset_index(drop=True)

,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,"[usability, external, editors]","[project, contain, resource, release, project,..."
1,2,NaN,"[open, repository, resources, doesnt]","[open, repository, resource, default, editor, ..."
2,3,NaN,"[indicate, deletion]","[deletion, indicator, viewer, subtle, vision, ..."
3,4,NaN,"[better, message, catch, resource]","[synchronize, project, repository, different, ..."
4,5,NaN,"[isharingmanager, share, inconsistent]","[gettingsetting, manage, resource, methods, is..."
...,...,...,...,...
70624,424656,NaN,"[fail, relate, config]","[feel, relate, need, change, relate, copy, bel..."
70625,424658,NaN,"[eclipse, kepler, ubuntu, jrockit]","[eclipse, kepler, ubuntu, oracle, jrockit, att..."
70626,424672,NaN,[display],"[button, dropdown, doesnt, button, result, but..."
70627,424714,NaN,[windows],"[start, test, notice, editor, windows, package..."


In [0]:
#Removing empty master_reports after data cleaning process
mr = pd.DataFrame()
count = 0
for i in range(len(master_reports)):
    if(len(master_reports.Description[i])>2):
        mr= mr.append(master_reports.loc[[i]])
        count=+1

In [0]:
#Saving all the master reports into a seperate csv file
master_reports.to_csv('drive/My Drive/duplicate_detection/master_reports.csv')